In [109]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [110]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yfin
import scipy.optimize as sco
import scipy.interpolate as sci

In [111]:
# Create a list of symbols
yfin.pdr_override()

symbols = ["XOM", "SHW", "JPM", "AEP", "UNH", "AMZN", "KO", "BA", "AMT", "DD", "TSN", "SLG"]
start_date = '2011-09-13'
end_date = '2023-04-19'

asset_data = pd.DataFrame()

for sym in symbols:
    try:
        data = pdr.get_data_yahoo(sym, start=start_date, end=end_date)
        asset_data[sym] = data['Adj Close']
    except Exception as e:
        print(e)

asset_data.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,XOM,SHW,JPM,AEP,UNH,AMZN,KO,BA,AMT,DD,TSN,SLG
Date,,,,,,,,,,,,
2011-09-13,45.134666,21.942066,23.339716,24.285767,40.272381,10.9765,24.170465,51.101665,42.487530,27.053284,14.016962,43.139282
2011-09-14,45.758308,22.627016,23.562407,24.501606,40.851410,11.1285,24.404823,51.248020,43.499519,27.670227,14.066639,44.577259
2011-09-15,46.621300,22.446146,24.287964,24.619337,42.412285,11.3390,24.842062,52.296883,44.061741,28.533970,14.472329,46.381275
2011-09-16,46.961464,22.348299,24.014977,24.540850,42.596912,11.9650,24.915516,53.158737,44.093864,29.099503,14.430928,47.047955
2011-09-19,46.426037,22.140728,23.339716,24.547398,41.917187,12.0845,24.656670,52.158669,44.342838,28.503109,14.273620,45.211273


In [112]:
# Compute daily simple returns
daily_returns = (
  asset_data.pct_change()
            .dropna(
              # Drop the first row since we have NaN's
              # The first date 2011-09-13 does not have a value since it is our cut-off date
              axis = 0,
              how = 'any',
              inplace = False
              )
)
# Examine the last 5 rows
daily_returns.tail(n = 5)

,XOM,SHW,JPM,AEP,UNH,AMZN,KO,BA,AMT,DD,TSN,SLG
Date,,,,,,,,,,,,
2023-04-12,-0.001040,-0.002210,-0.000156,-0.000848,0.000422,-0.020917,0.001758,0.000094,0.004183,0.002122,-0.007574,-0.047289
2023-04-13,0.004686,0.007397,0.003813,-0.003077,0.009670,0.046714,0.007338,0.005887,0.005293,0.006212,0.001786,-0.002140
2023-04-14,0.002419,-0.007914,0.075510,-0.015643,-0.027440,0.001074,-0.001584,-0.055621,-0.018595,-0.011365,-0.014103,-0.016724
2023-04-17,-0.011633,0.022380,0.007929,0.008541,-0.012583,0.002244,0.006503,0.016459,0.009377,0.009793,0.010194,0.066725
2023-04-18,0.019529,-0.002731,0.011228,-0.008790,-0.001722,-0.004283,0.001576,0.016290,-0.003113,0.006746,-0.009277,0.001635


In [113]:
daily_returns.mean() * 252

XOM     0.116142
SHW     0.236113
JPM     0.195150
AEP     0.135006
UNH     0.250139
AMZN    0.247522
KO      0.099210
BA      0.190968
AMT     0.165535
DD      0.131435
TSN     0.166280
SLG     0.009833
dtype: float64

In [114]:
daily_returns.cov() * 252

,XOM,SHW,JPM,AEP,UNH,AMZN,KO,BA,AMT,DD,TSN,SLG
XOM,0.067790,0.020221,0.041429,0.014510,0.025209,0.019043,0.019423,0.048370,0.016581,0.041991,0.024494,0.044544
SHW,0.020221,0.065693,0.030411,0.016252,0.025765,0.029149,0.018564,0.034439,0.026227,0.035783,0.022609,0.032371
JPM,0.041429,0.030411,0.079136,0.015232,0.032712,0.028473,0.022936,0.057729,0.024159,0.053813,0.027559,0.055601
AEP,0.014510,0.016252,0.015232,0.038892,0.017358,0.011545,0.019157,0.020207,0.024527,0.017066,0.013941,0.026653
UNH,0.025209,0.025765,0.032712,0.017358,0.063266,0.024088,0.018670,0.034823,0.023408,0.030757,0.019994,0.031287
AMZN,0.019043,0.029149,0.028473,0.011545,0.024088,0.109607,0.014184,0.035818,0.025561,0.032193,0.016044,0.027907
KO,0.019423,0.018564,0.022936,0.019157,0.018670,0.014184,0.031187,0.027766,0.021159,0.022311,0.017273,0.028467
BA,0.048370,0.034439,0.057729,0.020207,0.034823,0.035818,0.027766,0.138833,0.027319,0.057745,0.034657,0.069105
AMT,0.016581,0.026227,0.024159,0.024527,0.023408,0.025561,0.021159,0.027319,0.056434,0.024153,0.015651,0.035151
DD,0.041991,0.035783,0.053813,0.017066,0.030757,0.032193,0.022311,0.057745,0.024153,0.094943,0.028304,0.051098


In [115]:
# Function for computing portfolio return
def portfolio_returns(weights):
    return (np.sum(daily_returns.mean() * weights)) * 252

In [116]:
# Function for computing standard deviation of portfolio returns
def portfolio_sd(weights):
    return np.sqrt(np.transpose(weights) @ (daily_returns.cov() * 252) @ weights)

In [117]:
# instantiate empty list containers for returns and sd
list_portfolio_returns = []
list_portfolio_sd = []
# For loop to simulate 5000 random weight vectors (numpy array objects)
for p in range(5000):
    # Return random floats in the half-open interval [0.0, 1.0)
    weights = np.random.random(size = len(symbols)) 
    # Normalize to unity
    # The /= operator divides the array by the sum of the array and rebinds "weights" to the new object
    weights /= np.sum(weights) 
    # Lists are mutable so growing will not be memory inefficient
    list_portfolio_returns.append(portfolio_returns(weights))
    list_portfolio_sd.append(portfolio_sd(weights))
    # Convert list to numpy arrays
    port_returns = np.array(object = list_portfolio_returns)
    port_sd = np.array(object = list_portfolio_sd)

In [118]:
# Max expected return
round(max(port_returns), 4)

0.2041

In [119]:
# Min expected return
round(min(port_returns), 4)

0.12

In [120]:
# Max sd
round(max(port_sd), 4)

0.2269

In [121]:
# Min sd
round(min(port_sd), 4)

0.1605

In [122]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [123]:
import plotly.express as px

# Create a dataframe with the portfolio returns and standard deviations
df = pd.DataFrame({'Portfolio Returns': port_returns, 'Portfolio Standard Deviations': port_sd})

# Calculate the Sharpe Ratio for each portfolio
df['Sharpe Ratio'] = df['Portfolio Returns'] / df['Portfolio Standard Deviations']

# Plot the trade-off diagram
fig = px.scatter(df, x='Portfolio Standard Deviations', y='Portfolio Returns', color='Sharpe Ratio',
                 labels={'Portfolio Standard Deviations': 'Portfolio Standard Deviation (Annualized)',
                         'Portfolio Returns': 'Expected Portfolio Return (Annualized)',
                         'Sharpe Ratio': 'Sharpe Ratio'},
                 hover_data=['Sharpe Ratio'], size=[1]*len(df),
                 title='Mean-Standard Deviation Diagram',
                 width=800, height=500)

fig.show()

In [124]:
# User defined Sharpe ratio function
# Negative sign to compute the negative value of Sharpe ratio
def sharpe_fun(weights):
    return - (portfolio_returns(weights) / portfolio_sd(weights))

In [125]:
# We use an anonymous lambda function
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

In [126]:
# This creates 12 tuples of (0, 1), all of which exist within a container tuple
# We essentially create a sequence of (min, max) pairs
bounds = tuple(
  (0, 1) for w in weights
)

In [127]:
# Repeat the list with the value (1 / 12) 12 times, and convert list to array
equal_weights = np.array(
  [1 / len(symbols)] * len(symbols)
)

In [128]:
# Minimization results
max_sharpe_results = sco.minimize(
  # Objective function
  fun = sharpe_fun, 
  # Initial guess, which is the equal weight array
  x0 = equal_weights, 
  method = 'SLSQP',
  bounds = bounds, 
  constraints = constraints
)

In [129]:
# Extract the weight composition array
max_sharpe_results["x"]

array([1.00977885e-16, 2.45145506e-01, 4.72077739e-03, 1.70613919e-01,
       3.36767004e-01, 1.56181095e-01, 0.00000000e+00, 9.11145810e-17,
       4.55941227e-04, 8.81580906e-17, 8.61157585e-02, 0.00000000e+00])

In [130]:
# Expected return
max_sharpe_port_return = portfolio_returns(max_sharpe_results["x"])
round(max_sharpe_port_return, 4)

0.2191

In [131]:
# Standard deviation
max_sharpe_port_sd = portfolio_sd(max_sharpe_results["x"])
round(max_sharpe_port_sd, 4)

0.1782

In [132]:
# Sharpe ratio
max_sharpe_port_sharpe = max_sharpe_port_return / max_sharpe_port_sd
round(max_sharpe_port_sharpe, 4)

1.2299

In [133]:
# Minimize sd
min_sd_results = sco.minimize(
  # Objective function
  fun = portfolio_sd, 
  # Initial guess, which is the equal weight array
  x0 = equal_weights, 
  method = 'SLSQP',
  bounds = bounds, 
  constraints = constraints
)

In [134]:
# Expected return
min_sd_port_return = portfolio_returns(min_sd_results["x"])
round(min_sd_port_return, 4)

0.1465

In [135]:
# Standard deviation
min_sd_port_sd = portfolio_sd(min_sd_results["x"])
round(min_sd_port_sd, 4)

0.151

In [136]:
# Sharpe ratio
min_sd_port_sharpe = min_sd_port_return / min_sd_port_sd
round(min_sd_port_sharpe, 4)

0.9706

In [137]:
# We use anonymous lambda functions
# The argument x will be the weights
constraints = (
  {'type': 'eq', 'fun': lambda x: portfolio_returns(x) - target}, 
  {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
)

In [138]:
# This creates 12 tuples of (0, 1), all of which exist within a container tuple
# We essentially create a sequence of (min, max) pairs
bounds = tuple(
  (0, 1) for w in weights
)

In [139]:
# Initialize an array of target returns
target = np.linspace(
    start=0.15, 
    stop=0.30,
    num=100
)
# instantiate empty container for the objective values to be minimized
obj_sd = []
# For loop to minimize objective function
for target in target:
    min_result_object = sco.minimize(
        # Objective function
        fun=portfolio_sd, 
        # Initial guess, which is the equal weight array
        x0=equal_weights, 
        method='SLSQP',
        bounds=bounds, 
        constraints=constraints
    )
    # Extract the objective value and append it to the output container
    obj_sd.append(min_result_object['fun'])
# End of for loop
# Convert list to array
obj_sd = np.array(obj_sd)
# Rebind target to a new array object
target = np.linspace(
    start=0.15, 
    stop=0.30,
    num=100
)

In [140]:
import pandas as pd
import plotly.express as px

# Annotations for maximal Sharpe ratio and minimum variance portfolio
annotation_data = pd.DataFrame({
    'x': [max_sharpe_port_sd, min_sd_port_sd],
    'y': [max_sharpe_port_return, min_sd_port_return],
    'type': ["Maximal Sharpe Ratio Portfolio", "Minimum Variance Portfolio"]
})

# Plot data
fig = px.scatter(
    x=port_sd, y=port_returns,
    color=port_returns/port_sd,
    title="Mean-Standard Deviation Diagram",
    labels={
        "x": "Portfolio Standard Deviation (Annualized)",
        "y": "Expected Portfolio Return (Annualized)"
    },
    range_x=[0.1, max(port_sd)*1.2],  
    range_y=[0.1, max(port_returns)*1.2],
    opacity=0.7,
    size=[5]*len(port_sd)
)

# Add efficient frontier
fig.add_trace(
    px.line(x=obj_sd, y=target, labels={"x": "Risk", "y": "Return"}, title="Efficient Frontier").data[0]
)

# Add maximal Sharpe ratio portfolio
fig.add_trace(
    px.scatter(
        x=[max_sharpe_port_sd],
        y=[max_sharpe_port_return],
        color=[max_sharpe_port_return/max_sharpe_port_sd],
        title="Maximal Sharpe Ratio Portfolio"
    ).data[0]
)

# Add minimum variance portfolio
fig.add_trace(
    px.scatter(
        x=[min_sd_port_sd],
        y=[min_sd_port_return],
        color=[min_sd_port_return/min_sd_port_sd],
        title="Minimum Variance Portfolio"
    ).data[0]
)

# Add annotations
fig.add_annotation(
    x=annotation_data['x'][0],
    y=annotation_data['y'][0],
    text=annotation_data['type'][0],
    showarrow=True,
    arrowhead=5,
    arrowsize=.5,
    ax=20,
    ay=-40
)

fig.add_annotation(
    x=annotation_data['x'][1],
    y=annotation_data['y'][1],
    text=annotation_data['type'][1],
    showarrow=True,
    arrowhead=5,
    arrowsize=.5,
    ax=20,
    ay=-40
)

# Add colorbar
fig.update_layout(
    coloraxis_colorbar=dict(title="Sharpe Ratio")
)

fig.show()

In [141]:
# Efficient portfolio returns
# The function np.argmin() returns the indices of the minimum value(s)
# Take a slice starting from the minimum variance portfolio and exclude previous elements
efficient_sd = obj_sd[np.argmin(a = obj_sd):]
# Take a slice starting from the same index and exclude previous elements
efficient_returns = target[np.argmin(a = obj_sd):]

In [142]:
# Sort the x and y arrays using np.argsort()
sort_indices = np.argsort(efficient_sd)
x_sorted = efficient_sd[sort_indices]
y_sorted = efficient_returns[sort_indices]

# Cubic spline interpolation
tck = sci.splrep(
  x=x_sorted,
  y=y_sorted,
  k=3
)

In [143]:
# Define functional approximation of efficient frontier
def f(x):
    return sci.splev(
        x=x,
        tck=tck,
        der=0
    )

# Define first derivative of the efficient frontier
def df(x):
    return sci.splev(
        x=x,
        tck=tck,
        der=1
    )

In [144]:
# System of equations
def sys_eq(p, r_f=0.01):
    # Equations
    eq1 = r_f - p[0]
    eq2 = p[1] - df(p[2])
    eq3 = r_f + df(p[2]) * p[2] - f(p[2])
    # Output values
    return eq1, eq2, eq3

In [145]:
# Solve the linear system
sol_set = sco.fsolve(
    # Equations to solve for
    func = sys_eq,
    # Initial guess for p
    # This can be determined by trial and error or from the plot above (may take more than one try)
    x0 = [0.05, 1, 0.15]
)

C:\Users\o1656\anaconda3\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning:

The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.



In [146]:
# Sanity check
np.round(
  sys_eq(
    p = sol_set
  ),
  4
)

array([0.   , 0.   , 0.007])

In [193]:
# Constraints
# The target return is now f(x) where x is taken from the solution set above
constraints = (
  {'type': 'eq', 'fun': lambda x: portfolio_returns(x) - f(x = sol_set[2])}, 
  {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
)
# Optimize
min_result_object_tangency = sco.minimize(
  # Objective function
  fun = portfolio_sd, 
  # Initial guess, which is the equal weight array
  x0 = equal_weights, 
  method = 'SLSQP',
  bounds = bounds, 
  constraints = constraints
)

In [194]:
# Expected return
tangency_port_return = portfolio_returns(min_result_object_tangency["x"])
round(tangency_port_return, 4)

0.2187

In [195]:
# Standard deviation
tangency_port_sd = portfolio_sd(min_result_object_tangency["x"])
round(tangency_port_sd, 4)

0.1778

In [196]:
# Sharpe ratio
tangency_port_sharpe = tangency_port_return / tangency_port_sd
round(tangency_port_sharpe, 4)

1.2299

In [197]:
# Capital market line
def cml(x):
    return sol_set[0] + sol_set[1] * x

In [198]:
# Standard deviations
cml_sd = np.linspace(
  start = 0, 
  stop = 0.25,
  num = 100
)
# Expected returns
cml_exp_returns = cml(x = cml_sd)

In [199]:
cml_sd

array([0.        , 0.00252525, 0.00505051, 0.00757576, 0.01010101,
       0.01262626, 0.01515152, 0.01767677, 0.02020202, 0.02272727,
       0.02525253, 0.02777778, 0.03030303, 0.03282828, 0.03535354,
       0.03787879, 0.04040404, 0.04292929, 0.04545455, 0.0479798 ,
       0.05050505, 0.0530303 , 0.05555556, 0.05808081, 0.06060606,
       0.06313131, 0.06565657, 0.06818182, 0.07070707, 0.07323232,
       0.07575758, 0.07828283, 0.08080808, 0.08333333, 0.08585859,
       0.08838384, 0.09090909, 0.09343434, 0.0959596 , 0.09848485,
       0.1010101 , 0.10353535, 0.10606061, 0.10858586, 0.11111111,
       0.11363636, 0.11616162, 0.11868687, 0.12121212, 0.12373737,
       0.12626263, 0.12878788, 0.13131313, 0.13383838, 0.13636364,
       0.13888889, 0.14141414, 0.14393939, 0.14646465, 0.1489899 ,
       0.15151515, 0.1540404 , 0.15656566, 0.15909091, 0.16161616,
       0.16414141, 0.16666667, 0.16919192, 0.17171717, 0.17424242,
       0.17676768, 0.17929293, 0.18181818, 0.18434343, 0.18686

In [200]:
cml_exp_returns

array([0.01      , 0.0130623 , 0.0161246 , 0.0191869 , 0.0222492 ,
       0.0253115 , 0.0283738 , 0.03143611, 0.03449841, 0.03756071,
       0.04062301, 0.04368531, 0.04674761, 0.04980991, 0.05287221,
       0.05593451, 0.05899681, 0.06205911, 0.06512141, 0.06818372,
       0.07124602, 0.07430832, 0.07737062, 0.08043292, 0.08349522,
       0.08655752, 0.08961982, 0.09268212, 0.09574442, 0.09880672,
       0.10186902, 0.10493133, 0.10799363, 0.11105593, 0.11411823,
       0.11718053, 0.12024283, 0.12330513, 0.12636743, 0.12942973,
       0.13249203, 0.13555433, 0.13861663, 0.14167893, 0.14474124,
       0.14780354, 0.15086584, 0.15392814, 0.15699044, 0.16005274,
       0.16311504, 0.16617734, 0.16923964, 0.17230194, 0.17536424,
       0.17842654, 0.18148885, 0.18455115, 0.18761345, 0.19067575,
       0.19373805, 0.19680035, 0.19986265, 0.20292495, 0.20598725,
       0.20904955, 0.21211185, 0.21517415, 0.21823646, 0.22129876,
       0.22436106, 0.22742336, 0.23048566, 0.23354796, 0.23661

In [201]:
#Efficient frontier and Capital market line
fig = px.scatter(
  x=port_sd, y=port_returns,
  color=port_returns/port_sd,
)

#Random portfolios
fig.add_trace(
  go.Scatter(
    x=obj_sd, y=target,
    mode="markers",
    marker=dict(size=12)
  )
)

#Capital market line
fig.add_trace(
  go.Scatter(
    x=cml_sd, y=cml_exp_returns,
    mode="lines",
    line=dict(color="#9467bd")
  )
)

fig.update_layout(
  title="Mean-Standard Deviation Diagram",
  xaxis=dict(title="Portoflio Standard Deviation (Annualized)", tickformat=".2%"),
  yaxis=dict(title="Expected Portfolio Return (Annualized)", tickformat=".2%"),
  annotations=[
    dict(
      x=tangency_port_sd,
      y=tangency_port_return,
      text="Tangency Portfolio \n Expected Return: 21.87% \n Volatility: 17.78%",
      xref="x",
      yref="y",
      showarrow=True,
      arrowhead=7,
      ax=20,
      ay=-40
    )
  ]
)

fig.show()